# 1. Data preprocessing

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
mao_sentences = []
for root, dirs, files in os.walk("../mao"):
    for file in files:
        if file.endswith(".txt"):
            print(os.path.join(root, file))
            with open(os.path.join(root, file), "r") as input:
                sentences = input.read().replace('?','.').replace('!','.').replace('\n','.')
                sentences = sentences.split(".")   #. denotes end of sentence
            mao_sentences.extend(sentences)
            
print(len(mao_sentences))
print(mao_sentences[0])

../mao/REPORT ON AN INVESTIGATION OF THE PEASANT MOVEMENT IN HUNAN.txt
../mao/WHY IS IT THAT RED POLITICAL POWER CAN EXIST IN CHINA.txt
../mao/ANALYSIS OF THE CLASSES IN CHINESE SOCIETY.txt
../mao/ON CORRECTING MISTAKEN IDEAS IN THE PARTY.txt
../mao/A SINGLE SPARK CAN START A PRAIRIE FIRE.txt
../mao/BE CONCERNED WITH THE WELL-BEING OF THE MASSES, PAY ATTENTION TO METHODS OF WORK.txt
1702
THE IMPORTANCE OF THE PEASANT PROBLEM


In [3]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

mao_df = pd.DataFrame(mao_sentences, columns = ['Sentence'])
mao_df.head()

,Sentence
0,THE IMPORTANCE OF THE PEASANT PROBLEM
1,During my recent visit to Hunan [1] I made a f...
2,In the thirty-two days from January 4 to Febr...
3,Many of the hows and whys of the peasant move...
4,I saw and heard of many strange things of whi...


In [4]:
mao_df['NumWords'] = mao_df['Sentence'].apply(lambda x: len(x.split()))

In [5]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = ''
    lower = texts.lower()
    no_punctuation = NON_ALPHANUM.sub(r' ', lower)
    no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
    return no_non_ascii
  
mao_df['Sentence2'] = mao_df['Sentence'].apply(normalize_texts)
#mao_df.head()
#mao_df['Sentence2'] = mao_df['Sentence2'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
mao_df['Sentence2'] = mao_df['Sentence2'].apply(lambda x: ' '.join([w for w in x.split()]))

In [6]:
mao_df = mao_df[mao_df.NumWords>=5]
mao_df = mao_df.reset_index(drop=True)
mao_df

,Sentence,NumWords,Sentence2
0,THE IMPORTANCE OF THE PEASANT PROBLEM,6,the importance of the peasant problem
1,During my recent visit to Hunan [1] I made a f...,25,during my recent visit to hunan 1 i made a fir...
2,In the thirty-two days from January 4 to Febr...,48,in the thirty two days from january to februar...
3,Many of the hows and whys of the peasant move...,24,many of the hows and whys of the peasant movem...
4,I saw and heard of many strange things of whi...,15,i saw and heard of many strange things of whic...
...,...,...,...
1167,All revolutionary cadres have a tremendous re...,7,all revolutionary cadres have a tremendous res...
1168,After this congress we must adopt effective m...,30,after this congress we must adopt effective me...
1169,We must create thousands of townships like Ch...,14,we must create thousands of townships like cha...
1170,They will be our strongholds,5,they will be our strongholds


# 2. Sentence Embedding

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('stsb-roberta-large')

In [8]:
sentence_embeddings = model.encode(mao_df.Sentence2)
sentence_embeddings.shape

(1172, 1024)

# 3. Norm Caculation

Caculate norm value in a small dataset

In [9]:
norm_test_df = mao_df[:3]
norm_test_df

,Sentence,NumWords,Sentence2
0,THE IMPORTANCE OF THE PEASANT PROBLEM,6,the importance of the peasant problem
1,During my recent visit to Hunan [1] I made a f...,25,during my recent visit to hunan 1 i made a fir...
2,In the thirty-two days from January 4 to Febr...,48,in the thirty two days from january to februar...


In [10]:
norm_test = sentence_embeddings[:3]
norm_test.shape

(3, 1024)

In [11]:
from numpy import linalg as LA
norm_test_df["norm"] = LA.norm(norm_test, axis=1)
norm_test_df

,Sentence,NumWords,Sentence2,norm
0,THE IMPORTANCE OF THE PEASANT PROBLEM,6,the importance of the peasant problem,28.648369
1,During my recent visit to Hunan [1] I made a f...,25,during my recent visit to hunan 1 i made a fir...,28.259842
2,In the thirty-two days from January 4 to Febr...,48,in the thirty two days from january to februar...,28.657330


In [12]:
norm_test_df.sort_values(by='norm', ascending=False,ignore_index=True)

,Sentence,NumWords,Sentence2,norm
0,In the thirty-two days from January 4 to Febr...,48,in the thirty two days from january to februar...,28.657330
1,THE IMPORTANCE OF THE PEASANT PROBLEM,6,the importance of the peasant problem,28.648369
2,During my recent visit to Hunan [1] I made a f...,25,during my recent visit to hunan 1 i made a fir...,28.259842


Caculate norm value in the whole dataset

In [13]:
mao_df["norm"] = LA.norm(sentence_embeddings, axis=1)
norm_df_descending = mao_df.sort_values(by='norm', ascending=False,ignore_index=True)
norm_df_descending

,Sentence,NumWords,Sentence2,norm
0,Much has also been done in other fields,8,much has also been done in other fields,30.689739
1,I saw and heard of many strange things of whi...,15,i saw and heard of many strange things of whic...,30.457001
2,""" As such people have seen better days and are...",36,as such people have seen better days and are n...,30.412542
3,Without the poor peasants there would be no r...,9,without the poor peasants there would be no re...,30.313114
4,"This change is now taking place, and it is an...",18,this change is now taking place and it is an i...,30.297270
...,...,...,...,...
1167,ON THE DISREGARD OF ORGANIZATIONAL DISCIPLINE,6,on the disregard of organizational discipline,24.996475
1168,The modern industrial proletariat numbers abo...,8,the modern industrial proletariat numbers abou...,24.992758
1169,""" The middle peasant would reply, ""There's no ...",9,the middle peasant would reply there s no hurry,24.607372
1170,40 yuan for the latter,5,0 yuan for the latter,24.486843


In [14]:
for index, row in norm_df_descending.iterrows():
    print(index, row["Sentence"])

0  Much has also been done in other fields
1  I saw and heard of many strange things of which I had hitherto been unaware
2 " As such people have seen better days and are now going downhill with every passing year, their debts mounting and their life becoming more and more miserable, they "shudder at the thought of the future"
3  Without the poor peasants there would be no revolution
4  This change is now taking place, and it is an important factor for the completion of the revolution
5  If the nation-wide revolutionary situation does not continue to develop but stagnates for a fairly long time, then it will be impossible for the small Red areas to last long
6  I have lived here for decades, tilling my land
7  At meetings, all participants should be encouraged to voice their opinions as fully as possible
8  Without these objectives, fighting loses its meaning and the Red Army loses the reason for its existence
9  If we attend to these problems, solve them and satisfy the needs of the m

682 " Only under the impact of all this are the rich peasants tardily joining the associations, [11] some paying fifty cents or a yuan for admission (the regular fee being a mere ten coppers), some securing admission only after asking other people to put in a good word for them
683  When the peasants' power grew till it matched that of the landlords, the magistrate took the position of trying to accommodate both the landlords and the peasants, accepting some of the peasant association's suggestions while rejecting others
684  This form of punishment more than any other makes the local tyrants and evil gentry tremble
685 " This opinion comes to the same thing as what the right-wingers are saying; according to both it is all right to have a peasant movement (the movement is already in being and no one dare say otherwise), but they say that the people running it are no good and they particularly hate those in charge of the associations at the lower levels, calling them "riffraff"
686  In 

# Coreference Resolution
Coreference Resolution aims at solving implicit references issues   
Explanation: https://medium.com/huggingface/state-of-the-art-neural-coreference-resolution-for-chatbots-3302365dcf30   
GitHub: https://github.com/huggingface/neuralcoref

In [35]:
!pip uninstall -y neuralcoref
!pip install --no-cache-dir  neuralcoref --no-binary neuralcoref
!pip install -U spacy==2.1.0
!python -m spacy download en

Found existing installation: neuralcoref 4.0
Uninstalling neuralcoref-4.0:
  Successfully uninstalled neuralcoref-4.0
     |████████████████████████████████| 368 kB 3.2 MB/s eta 0:00:01
Skipping wheel build for neuralcoref, due to binaries being disabled for it.
    Running setup.py install for neuralcoref ... done
  Using cached spacy-2.3.7-cp37-cp37m-macosx_10_9_x86_64.whl (10.2 MB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached srsly-1.0.5-cp37-cp37m-macosx_10_9_x86_64.whl (177 kB)
  Using cached thinc-7.4.5-cp37-cp37m-macosx_10_9_x86_64.whl (969 kB)


  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.2
    Uninstalling srsly-2.4.2:
      Successfully uninstalled srsly-2.4.2
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.6
    Uninstalling catalogue-2.0.6:
      Successfully uninstalled catalogue-2.0.6
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.13
    Uninstalling thinc-8.0.13:
      Successfully uninstalled thinc-8.0.13
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.0
    Uninstalling spacy-3.2.0:
      Successfully uninstalled spacy-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.2.0 requires spacy<3.3.0,>=3.2.0, but you have spacy 2.3.7 which is incompatible.
     |████████████████████████████████| 12.0 MB 3.4 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=e

In [15]:
import en_core_web_sm
import spacy
nlp = spacy.load('en')

In [16]:
import neuralcoref
neuralcoref.add_to_pipe(nlp)

In [17]:
doc = nlp(u'My sister has a dog. She loves him.')
doc._.coref_resolved

'My sister has a dog. My sister loves a dog.'

use this algorithm on our data

In [18]:
import os
articles = []
modified_articles = []
for root, dirs, files in os.walk("../mao"):
    for file in files:
        if file.endswith(".txt"):
            print(os.path.join(root, file))
            with open(os.path.join(root, file), "r") as input:
                sentences = input.read().replace('?','.').replace('!','.').replace('\n','.')
                articles.append(sentences)
                #print(sentences)
                doc = nlp(sentences)
                modified_articles.append(doc._.coref_resolved)
                print(doc._.coref_resolved)
            print()

../mao/REPORT ON AN INVESTIGATION OF THE PEASANT MOVEMENT IN HUNAN.txt
THE IMPORTANCE OF THE PEASANT PROBLEM.During my recent visit to Hunan [1] I made a first-hand investigation of conditions in the five counties of Hsiangtan, Hsianghsiang, Hengshan, Liling and Changsha. In the thirty-two days from January 4 to February 5, I called together fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement, and I listened attentively to fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement reports and collected a great deal of material. Many of the hows and whys of the peasant movement were the exact opposite of what the gentry in Hankow and Changsha are saying. I saw and heard of many strange things of which I had hitherto been unaware. I believe the same is true of many other places, too. All talk directed agains

The present regime of the new warlords of the Kuomintang remains a regime of the comprador class in the cities and the landlord class in the countryside; it is a regime which has capitulated to imperialism in it foreign relations and which at home has replaced the old warlords with new ones, subjecting the working class and the peasantry to an even more ruthless economic exploitation and political oppression. The bourgeois-democratic revolution which started in Kwangtung Province had gone only halfway when the comprador and landlord classes usurped the leadership and immediately shifted the leadership on to the road of counter-revolution; throughout the country the workers, the peasants, the other sections of the common people, and even the bourgeoisie, [1] have remained under counter-revolutionary rule and obtained not the slightest particle of political or economic emancipation...Before their capture of Peking and Tientsin, the four cliques of the old warlords, Chiang Kai-shek, the K

Who are our enemies. Who are our friends. This is a question of the first importance for the revolution. The basic reason why all previous revolutionary struggles in China achieved so little was all previous revolutionary struggles in China failure to unite with real friends in order to attack real enemies. A revolutionary party is the guide of the masses, and no revolution ever succeeds when A revolutionary party leads A revolutionary party astray. To ensure that we will definitely achieve success in we and will not lead the masses astray, we must pay attention to uniting with we real friends in order to attack we real enemies. To distinguish real friends from real enemies, we must make a general analysis of the economic status of the various classes in Chinese society and of their respective attitudes towards the revolution...What is the condition of each of the classes in Chinese society...The landlord class and the comprador class.[1] In economically backward and semi-colonial Chin

ON THE PURELY MILITARY VIEWPOINT.The purely military viewpoint is very highly developed among a number of comrades in the Red Army. It manifests It as follows:..1. These comrades regard military affairs and politics as opposed to each other and refuse to recognize that military affairs are only one means of accomplishing political tasks. Some even say, "If you are good militarily, naturally you are good politically; if you are not good militarily, you cannot be any good politically"--this is to go a step further and give military affairs a leading position over politics...2. They think that the task of the Red Army like that of the Red Army, is merely to fight. They do not understand that the Red Army is an armed body for carrying out the political tasks of the revolution. Especially at present, the Red Army should certainly not confine the Red Army to fighting; besides fighting to destroy the enemy's military strength, the Red Army should shoulder such important tasks as doing propaga

Some comrades in our Party still do not know how to appraise the current situation correctly and how to settle the attendant question of what action to take. Though Some comrades in our Party believe that a revolutionary high tide is inevitable, Some comrades in our Party do not believe it to be imminent. Therefore, Some comrades in our Party disapprove of the plan to take Kiangsi and only approve of roving guerrilla actions in the three areas on the borders of Fukien, Kwangtung and Kiangsi; at the same time, as Some comrades in our Party do not have a deep understanding of what it means to establish Red political power in the guerrilla areas, Some comrades in our Party do not have a deep understanding of the idea of accelerating the nation-wide revolutionary high tide through the consolidation and expansion of Red political power. Some comrades in our Party seem to think that, since the revolutionary high tide is still remote, it will be labour lost to attempt to establish political p

There are two questions which comrades have failed to stress during the discussion and which, I feel, should be dealt with...The first concerns the well-being of the masses...Our central task at present is to mobilize the broad masses to take part in the revolutionary war, overthrow imperialism and the Kuomintang by means of such war, spread the revolution throughout the country, and drive imperialism out of China Anyone who does not attach enough importance to this central task is not a good revolutionary cadre. If our comrades really comprehend this central task and understand that the revolution must at all costs be spread throughout the country, then our comrades should in no way neglect or underestimate the question of the immediate interests, the well-being, of the broad masses. For the revolutionary war is a war of the masses; the revolutionary war can be waged only by mobilizing the masses and relying on the masses...If we only mobilize the people to carry on the revolutionary 

Create a dictionary, the key is the original sentence and the value is modified sentence

In [19]:
coref_resolution_dict = {}

for i in range(len(articles)):
    origin_sentences = articles[i].split('.')
    modified_sentences = modified_articles[i].split('.')
    for j in range(len(origin_sentences)):
        if(origin_sentences[j] != modified_sentences[j]):
            coref_resolution_dict[origin_sentences[j]] = modified_sentences[j]
            print(origin_sentences[j] + '\n' + modified_sentences[j])
            print()

 In the thirty-two days from January 4 to February 5, I called together fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement, and I listened attentively to their reports and collected a great deal of material
 In the thirty-two days from January 4 to February 5, I called together fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement, and I listened attentively to fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement reports and collected a great deal of material

 They will smash all the trammels that bind them and rush forward along the road to liberation
 several hundred million peasants will smash all the trammels that bind several hundred million peasants and rush forward along the road to liberation

 They

Print sentences in descending order of Norm value

In [20]:
for index, row in norm_df_descending.iterrows():
    if(coref_resolution_dict.get(row["Sentence"])):
        print(index,"CHANGED", coref_resolution_dict.get(row["Sentence"]))
    else:
        print(index, row["Sentence"])

0  Much has also been done in other fields
1  I saw and heard of many strange things of which I had hitherto been unaware
2 CHANGED " As such people have seen better days and are now going downhill with every passing year, such people debts mounting and such people life becoming more and more miserable, such people "shudder at the thought of the future"
3 CHANGED  Leadership by The poor peasants is absolutely necessary
4 CHANGED  The Revolution of 1911 [3] did not bring about this change, hence The Revolution of 1911 [3] failure
5  If the nation-wide revolutionary situation does not continue to develop but stagnates for a fairly long time, then it will be impossible for the small Red areas to last long
6 CHANGED  A tolerably well-behaved one would say, "Peasant association
7 CHANGED  At meetings, all participants should be encouraged to voice all participants opinions as fully as possible
8 CHANGED  Without these objectives, fighting loses fighting meaning and the Red Army loses the re

599 CHANGED  The poor peasants have beaten down the local tyrants and evil gentry, big and small, and kept The poor peasants underfoot
600  The peasants work out fines for such offences as irregularities revealed by the checking of accounts, past outrages against the peasants, current activities which undermine the peasant associations, violations of the ban on gambling and refusal to surrender opium pipes
601 CHANGED  Although the appraisals and directives of the Central Committee has not made any correction of the letter to the Red Army, the Red Army subsequent directives have not been couched in the same pessimistic tone and the Red Army views on the Red Army operations now coincide with ours
602 CHANGED  By the end of January a total membership of 1,367,727 must have reached at least two million
603 CHANGED  In the case of Yen Jung-chiu of Hsiangtan, the peasants and other sections of the people compelled the county magistrate to agree to hand the county magistrate over, and the pe

1039 CHANGED  the membership of the peasant associations the peasant associations jumped to two million and the masses directly under the membership of the peasant associations leadership increased to ten million
1040  Thirdly, one will understand the correctness of the slogan of workers' and peasants' democratic political power
1041  In Chekiang, there is only a small provincial force under Chiang Po-cheng
1042 CHANGED  Such township governments are absolutely different from the Tingchow Municipal Government with the Tingchow Municipal Government bureaucratic methods of leadership
1043 CHANGED  We are not clear about the situation in Chekiang, but Party's organizational and mass base is somewhat better in Kiangsi and Fukien than in Hunan
1044 CHANGED  It is against interpreting the Kuomintang's Principle of the People's Livelihood according to the theory of class struggle, and the Kuomintang opposes the Kuomintang's alliance with Russia and the admission of Communists [6] and left-win

Try to do embedding on modified data again, and calculate the Norm value

In [26]:
modified_articles_sentences = []
for article in modified_articles:
    modified_articles_sentences.extend(article.split('.'))
    
modified_articles_sentences

['THE IMPORTANCE OF THE PEASANT PROBLEM',
 'During my recent visit to Hunan [1] I made a first-hand investigation of conditions in the five counties of Hsiangtan, Hsianghsiang, Hengshan, Liling and Changsha',
 ' In the thirty-two days from January 4 to February 5, I called together fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement, and I listened attentively to fact-finding conferences in villages and county towns, which were attended by experienced peasants and by comrades working in the peasant movement reports and collected a great deal of material',
 ' Many of the hows and whys of the peasant movement were the exact opposite of what the gentry in Hankow and Changsha are saying',
 ' I saw and heard of many strange things of which I had hitherto been unaware',
 ' I believe the same is true of many other places, too',
 ' All talk directed against the peasant movement must be speedily set r

In [27]:
len(modified_articles_sentences)

1715

In [35]:
mao_df = pd.DataFrame(modified_articles_sentences, columns = ['modified_sentence'])
mao_df.head()

,modified_sentence
0,THE IMPORTANCE OF THE PEASANT PROBLEM
1,During my recent visit to Hunan [1] I made a f...
2,In the thirty-two days from January 4 to Febr...
3,Many of the hows and whys of the peasant move...
4,I saw and heard of many strange things of whi...


In [36]:
mao_df['NumWords'] = mao_df['modified_sentence'].apply(lambda x: len(x.split()))

In [37]:
mao_df = mao_df[mao_df.NumWords>=5]
mao_df = mao_df.reset_index(drop=True)
mao_df

,modified_sentence,NumWords
0,THE IMPORTANCE OF THE PEASANT PROBLEM,6
1,During my recent visit to Hunan [1] I made a f...,25
2,In the thirty-two days from January 4 to Febr...,68
3,Many of the hows and whys of the peasant move...,24
4,I saw and heard of many strange things of whi...,15
...,...,...
1176,All revolutionary cadres have a tremendous re...,7
1177,After this congress we must adopt effective m...,30
1178,We must create thousands of townships like Ch...,14
1179,thousands of townships like Changkang and sco...,15


In [38]:
sentence_embeddings = model.encode(mao_df['modified_sentence'])
sentence_embeddings.shape

(1181, 1024)

In [39]:
mao_df["norm"] = LA.norm(sentence_embeddings, axis=1)
norm_df_descending = mao_df.sort_values(by='norm', ascending=False,ignore_index=True)
norm_df_descending

,modified_sentence,NumWords,norm
0,Much has also been done in other fields,8,30.689739
1,I saw and heard of many strange things of whi...,15,30.457001
2,Without The poor peasants there would be no r...,9,30.313114
3,"I have lived here for decades, tilling my land",9,30.085690
4,That is why the people are particularly good ...,9,30.050821
...,...,...,...
1176,The modern industrial proletariat numbers abo...,8,24.992758
1177,"Intensify education, criticize incorrect idea...",13,24.960115
1178,Of organizations with a total membership of 1...,38,24.698273
1179,By the end of January a total membership of 1...,17,24.476740


In [40]:
for index, row in norm_df_descending.iterrows():
    print(index, row["modified_sentence"])

0  Much has also been done in other fields
1  I saw and heard of many strange things of which I had hitherto been unaware
2  Without The poor peasants there would be no revolution
3  I have lived here for decades, tilling my land
4  That is why the people are particularly good fighters
5  The wooden bridge over there is too narrow and people may fall off
6  There are special reasons for this unusual phenomenon
7  This is a most serious and vital struggle
8 Some of the peasants can also recite Dr
9  If the nation-wide revolutionary situation does not continue to develop but stagnates for a fairly long time, then it will be impossible for the small Red areas to last long
10  Unless the bridge or boat problem is solved, it is idle to speak of crossing the river
11 " As such people have seen better days and are now going downhill with every passing year, such people debts mounting and such people life becoming more and more miserable, such people "shudder at the thought of the future"
12  

757  Those who fled to Changsha are only of the fourth rank, and the peasantry, having many eyes and ears, can easily track Those who fled to Changsha down
758  In Hsianghsiang County, each family in Yangtang is allowed to keep only three, and in Futienpu, Hengshan County
759  If so, we must be with so, arouse so enthusiasm and initiative, be concerned with so well-being, work earnestly and sincerely in so interests and solve all so problems of production and everyday life--the problems of salt, rice, housing, clothing, childbirth, etc
760  This was possible because the policies of the Communist Party organizations (local and army) in the Hunan-Kiangsi border area were correct
761  In the present chaotic situation we can lead the masses only by positive slogans and a positive attitude
762 In the Ching Dynasty, the household census compiled by the local authorities consisted of a regular register and "the other" register, the former for honest people and the latter for burglars, bandits